<a href="https://colab.research.google.com/github/MWSteams/4590FSP/blob/master/Rapid_Mix_Mechanical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rapid Mix Mechanical

Authors: Lynn Li, Allison Tran

[/Monroe/]: # (Well done! Great start. In addition to the vendors I'd suggest seeing if you can get any information from the world's biggest water treatment plants to see what other plants of this scale use for rapid mix. This would include the Cottonwood plant that the Fairmont plant will be eliminating and the LA filtration plant. I don't think that you need to know what the coagulant dose will be or what contaminants will be removed in order to design the rapid mix. The big question is whether the rapid mix is supposed to do something in addition to getting the coagulant equally mixed with water before it divides into multiple treatment trains.)

In [1]:
!pip install aguaclara
!pip install molmass

     |████████████████████████████████| 92kB 2.4MB/s 
     |████████████████████████████████| 163kB 8.7MB/s 
  Created wheel for aguaclara: filename=aguaclara-0.2.0-cp36-none-any.whl size=99623 sha256=43c855946a59267dbf9520c907c38e7170f99321355609dacc4a66951352c85c
  Stored in directory: /root/.cache/pip/wheels/34/e9/34/0d3722243083a982623290c205c14a08b54ed290ee29219fd2
  Created wheel for pint: filename=Pint-0.8.1-py2.py3-none-any.whl size=134156 sha256=3eb5ce108aa696f0fbcd2b52833743e207d3d5c973808b66f49327041d735ada
  Stored in directory: /root/.cache/pip/wheels/b4/56/36/7c4d6a4e3660e233b95e3b0482551915cb184abca0f93ee1dd
Successfully built aguaclara pint
     |████████████████████████████████| 71kB 2.1MB/s 


In [3]:
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import molmass as mol
!wget https://raw.githubusercontent.com/MWSteams/4590FSP/master/Fairmont.py
import Fairmont as fmt

--2020-04-18 14:56:49--  https://raw.githubusercontent.com/MWSteams/4590FSP/master/Fairmont.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1604 (1.6K) [text/plain]
Saving to: ‘Fairmont.py’

Fairmont.py         100%[===================>]   1.57K  --.-KB/s    in 0s      

2020-04-18 14:56:49 (29.0 MB/s) - ‘Fairmont.py’ saved [1604/1604]



# Task Definition

Our task was to evaluate the process of mechanical rapid mix, methods and strategies to optimize this process, and its purpose and effectiveness as part of the overall water treatment process. In a standard treatment train, rapid mix occurs between coagulant addition and flocculation, and provides an enhanced method of distributing the coagulant evenly through the water. Unfortunately, not much is known about rapid mix and how useful it truly is to the overall treatment train. Therefore our mechanical rapid mix will be aimed to reach a target energy dissipation rate. In turn, we hope that this will result in effectively covering all particles with coagulant. By modifying certain constraints and variables, our goal is to develop code for rapid mix that will accurately predict the turbidity, concentration of dissolved organic matter (DOM), and concentration of arsenic of the water output.

# Executive Summary 

At this time we cannot make a reccomendation because we aren't sure which contaminant will control the coagulant dosage and therefore we aren't sure which variable(s) we want to optimize in rapid mix. However, we are sure that energy dissipation will be important as well as how effectively the coagulant coats the particles.

[/Monroe/]: # (Mixing coagulant uniformly with the raw water before splitting the flow into multiple treatment trains might be one of the goals of rapid mix in this plant. I suspect that your design will not be influenced in any way by knowing the contaminant that controls the coagulant dosage because we don't have a physics-based model for the design of the rapid mix. See https://aguaclara.github.io/Textbook/Rapid_Mix/RM_Derivations.html#turbulent-large-scale-eddies for mixing length estimates. Note that you would want to include a safety factor on these equations.)


# Constraints

* Relevant Regulations
    * The rapid mix step needs to remove partial amounts of turbidity, DOM, and arsenic so that the water is up to drinking standards by the end of the treatment train 
    * We want to use this unit process to remove as much arsenic as possible
[/Monroe/]: # (Rapid Mix doesn't remove anything!)
* Client Expectations
    * Varying rapid mix systems have varying mixing times which then affects the plant's efficiency 
[/Monroe/]: # (Not clear if mixing times influence plant efficiency)
* Site and Budget
    * Varying rapid mix systems will use varying amounts of energy which then affects monthly electricity bills 
[/Monroe/]: # (Not clear what you mean here)
    * Rapid mix mechanical is one of the most energy intensive units in treatment train
    * Statics mixers are expensive (MWH)
    
* Process Capabilities Including Process Design Equations
    * Residence time affects the volume of water that can be treated at once 
    
[/Monroe/]: # (Residence time sets the required VOLUME of the unit. Calculate this for a range of designs.)
    * "Rapid blending must be achieved with water treatment chemicals that involve irreversible competitive consecutive reactions." (MWH) 
[/Monroe/]: # (Be skeptical of this statement.)
* Ease of Use
    * Static mixers are simple and reliable but they are prone to clogging (MWH)
* Operation Requirements
    * If there are multiple treatment trains, parallel treatment trains must receive the same concentration of coagulant; coagulant musst be mixed equally
    * Removing arsenic with an iron based coagulant will require a 
    * The optimal pH for aluminum coagulant nanoparticle formation is between pH of 6.5 and 8.5. Adding coagulant usually reduces pH so somtimes it is necessary to add a base to increase the pH of the raw water
* Maintenance Requirements
    * Needs to be easy to clean
    * Parts that can be easily bought and replaced

[/Monroe/]: # (A big design question is whether the rapid mix units are before or after the flow splitting to treatment trains.)

[/Monroe/]: # (I'm not sure, but I believe the table below is for a very unusual case where the coagulant is at concentrations below its solubility limit. We certainly don't attempt to have mixing complete in 0.3 seconds in AguaClara plants and some plant operators for conventional water treatment plants have turned off the rapid mix units and not seen a change in performance.)

[/Monroe/]: # (Quote from Benjamin and Lawler [https://newcatalog.library.cornell.edu/catalog/8516026] page 546: "While much research remains to be done on the interactions of chemicals, paricles, and mixing, the emerging picture at this point is tha ta high degree of turbulence leads to more efficient chemical use for destabilization by adsorption and charge neutralization, whereas destabilization by precipitation is less sensitive to the mixing conditions." My understanding is that drinking water treatment plants use destabilization by precipitation; the only mechanism that we have been able to replicate in our labs here at Cornell. We now recognize that it isn't destabilization, but rather is simply that the coagulant nanoparticles are sticky.)

See [new resources](https://raw.githubusercontent.com/MWSteams/4590FSP/master/resources.md) for information on rapid mix and the Cottonwood plant.

![Table 6-6](https://github.com/MWSteams/4590FSP/raw/Rapid_Mix_Mechanical/table_6-6.PNG)


# Physics

Briefly describe the physics that will determine whether your portion of the design succeeds or fails. Present relevant equations describing those physical laws and explain how you will use those concepts to guide your design.

$$G=\frac{\Delta v}{\Delta h}$$

where $G$ is the velocity gradient, $\Delta v$ is the difference in velocity between two points on two different streamlines and $\Delta h$ is the distance between the streamlines.

G is a measure of how quickly one point of water along one streamline moves in comparison to another point on a different streamline.

$$\varepsilon = G^2 \nu$$ 

where $\epsilon$ is the energy dissipation rate, $G$ is the velocity gradient, and $\nu$ is the kinematic viscosity of water.

It is hypotheiszed that it's ultimately velocity gradients that transport coagulant nanoparticles to the surface of particles.

$$P = \rho \bar G^2 \nu Q \theta$$ 

where $P$ is power input required, $\rho$ is density of water, $Q$ is the flow rate, and $\theta$ is the hydraulic residence time.

This equation can be used to estimate electrical motor sizing requirements for mechanical rapid mix units.

$$t_{coagulant, \, application} = \frac{2.3p C_{CN} \Lambda_{Clay}^2}{\pi G k \, d_{Clay}  L_{Diff_{CN}} }$$

where $t$ is time required to reach a target efficiency of application of coagulant nanoparticles to clay, $pC_{CN}$ describes the removal efficiency of other contaminants, and $\Lambda_{clay}$ is the average spacing of clay particles.

[/Monroe/]: # (The previous equation needs to be combined with an estimate of the diffusion length scale for the coagulant nanoparticles. In any case that equation strongly suggests that the coagulant nanoparticles are not transported to the clay surfaces in the short amount of time spent in the rapid mix.)

[/Monroe/]: # (Another equation is the eddy turnover time
https://aguaclara.github.io/Textbook/Rapid_Mix/RM_Derivations.html#equation-rapid-mix-rm-derivations-38)

# Table Listing Potential Strategies to Meet the Constraints and Treatment Objectives

| strategies | advantages | disadvantages | data needed for decision |
| ---------- | ---------- | ---------- | ---------- |
| Reducing energy used to power mechanical mixers | Lower cost, lower electricity usage | Less power to mixers affects energy dissipation, velocity, residence time, and more | Power usage typical for mechanical rapid mix |
| Modifying coagulant dosage to ensure removal of arsenic | Meet water treatment plant standards for arsenic removal | Rapid mix can be expensive (coagulant and electricity to power mixers) | Coagulant dosage needed to remove certain concentration of arsenic, cost of powering rapid mixers
| Modifying coagulant dosage to ensure even distribution | Increased coagulant will remove more contaminant (arsenic) | Increased coagulant will be more expensive | Type of coagulant used, velocity gradient, ideal dosage, residence time of mixer |

![](https://github.com/MWSteams/4590FSP/raw/Rapid_Mix_Mechanical/pump1.PNG)
![](https://github.com/MWSteams/4590FSP/raw/Rapid_Mix_Mechanical/pump2.PNG)
![](https://github.com/MWSteams/4590FSP/raw/Rapid_Mix_Mechanical/pump3.PNG)

[(a) pumped flash mixer, (b) in-line static mixer, and (c) in-line Venturi mixer]

# Vendors and Contact Information File

[/Monroe/]: # (Excellent work finding some suppliers)

In [4]:
vendors = pd.read_csv('https://raw.githubusercontent.com/MWSteams/4590FSP/master/vendors.txt',sep='\t')
vendors

,item description,vendor,contact first name,contact last name,vendor website,product info,date of contact,who made the contact,notes
0,plate settlers,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/literaturemedia/...,4/16/2020,Ian Cullings,NaN
1,sludge removal system,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/products/hoseles...,4/16/2020,Ian Cullings,NaN
2,plate settlers,Monore Environmental,still,pending,https://www.monroeenvironmental.com/,https://www.monroeenvironmental.com/water-and-...,4/16/2020,Ian Cullings,NaN
3,sludge removal system,Monore Environmental,still,pending,https://www.monroeenvironmental.com/,http://www.monroeenvironmental.com/water-and-w...,4/16/2020,Ian Cullings,NaN
4,plate settler with hopper,Parkson,still,pending,https://www.parkson.com/,https://www.parkson.com/products/lamella-ecoflow,4/16/2020,Ian Cullings,Includes sloped sludge hopper in design
5,rapid mixer (mech),Dynamix,still,pending,https://www.dynamixinc.com,https://www.dynamixinc.com/industries/waste-wa...,4/16/2020,Lynn Li,NaN
6,rapid mixer (mech),Meurer Research,still,pending,http://www.meurerresearch.com,http://www.meurerresearch.com/products/mixer/,4/16/2020,Lynn Li,NaN
7,rapid mixer (mech),Philadelphia Mixing Solutions,still,pending,https://www.philamixers.com,https://www.philamixers.com/industries/water-w...,4/16/20,Lynn Li,NaN


# Shared Assumptions

See this [python file, Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py)

Use this file to hold parameters that are in the SOW or elsewhere that
influence the design of the entire plant. This provides an easy mechanism
for all of us to share the same assumptions. Read the SOW to find sections that relate to your part of the design and enter all pertinent variables into [Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py) and then push those changes to github.

We are creating our model with the assumption that the main constituents for removal by rapid mix are turbidity, DOM, and arsenic. 

[/Monroe/]: # (Rewrite this now that you recall that rapid mix doesn't remove anything.)

We will be able to compile a list of pertinent variables once we begin the code for the process and have determined which variables will be needed from the file. 

[/Monroe/]: # (You can also create a range of rapid mix volumes, power requirements, and annual operating costs for standard designs for rapid mix units. Those preliminary designs are trivial and will help give insight for goal of reducing the energy footprint of this facility.)

# Major Decisions

Write several paragraphs describing the major decisions that you will need to make in order to create a design and outline a strategy for how you will obtain the necessary information to make those decisions.

# Next Steps and Challenges

The greatest challenge for us will be to figure out which contaminant (turbidity, DOM, or arsenic) concentration will determine/control the coagulant dose amount. More research needs to be done to better understand how each contaminant reacts to the iron based coagulant and the fluids knowledge behind how they would move in a rapid mix.

Hopefully we will be able to find some case studies that include quantitative data in order to move our deisgn forward. We will also be looking at previous AguaClara rapid mix mechanical designs.

# Design

Begin writing the code to start your design. Define flow rates, mass rates, energy requirements, and any readily calculated dimensions, velocities, flows, etc.

In [0]:
print('The average flow to be treated at Fairmont is',fmt.q_ave.to(u.Mgal/u.day),'or')
print('The maximum flow to be treated at Fairmont is',fmt.q_max.to(u.Mgal/u.day))
specific_energy = (fmt.electricity_power/fmt.q_ave).to(u.J/u.L)
temp = 15 * u.degC
specific_potential_energy = (specific_energy / u.gravity / ac.density_water(temp)).to(u.m)
fsp_flow = 480*(u.Mgal/u.day)
# G = 
print('The average amount of energy used per liter of water is expected to be',specific_energy,'or',specific_potential_energy,'of potential energy')

# References

https://aguaclara.github.io/Textbook/Rapid_Mix